

***
### Web Scraping from Indeed.com & H1Bdata.info
*This is a script to automate job searching process for people need sponsorships. It will save job seekers plenty of time by filtering for job postings that require certain skills and filtering for corresponding employers having historical sponsorship information.*
<div style="text-align: right"><font color='pink'>Helena Li</font></div>
<div style="text-align: right"><font color='pink'>created on 3/15/2021</font></div>

***

#### import packages

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import random
import time
from colorama import Fore
from tqdm import tqdm

#### input `location`, `position`, `time posted till today`, `number of pages`, `key words` for web scraping

In [2]:
## input customized values
location = 'United States'
position = 'Data Analyst'
time_post = '7'
## try to limit the page_num to 4 or less, to prevent from being blocked
page_num = 4

## format input values
location = location.capitalize()
position = position.lower()
location_link = location.replace(' ','+')
position_link = position.replace(' ','+')

## input customized values
## key_word list is for both the key words you are looking for in the job description, and key words serve as a red flag (require U.S. citizenship etc.)
key_word = ['SQL','Python','Tableau','Power BI', 'Looker', 'Analytics','Machine Learning','Statistics','Excel','A/B Test','hypothesis test','Model','U.S. Citizenship','contract']

#### scrape information from indeed.com

In [3]:
link = []
title = []
skill = []
company = []

## Adding tqdm() function to see processing bar
for i in tqdm(range(0,page_num)):
    start_num = i*10
    result = requests.get('https://www.indeed.com/jobs?q='+position_link+'&l='+location_link+'&fromage='+time_post+'start='+str(start_num))
    src = result.content
    soup = BeautifulSoup(src,'html.parser')
    td = soup.find_all("td",{"id":"resultsCol"})
    html = td[0].find_all("a",{"target":"_blank"})

    for h in tqdm(html):
        if re.findall('jobtitle turnstileLink',str(h)):
            new_link = 'https://www.indeed.com' + h.get('href')
            link.append(new_link)
            title.append(h.get('title'))
            requests.get(new_link)
            web = requests.get(new_link)
            src_web = web.content
            soup_web = BeautifulSoup(src_web,'html.parser')
            div = soup_web.find_all("div",{"class":"icl-u-lg-mr--sm icl-u-xs-mr--xs"})
            company.append(re.findall(r'>([A-Za-z0-9 -]+).*</div>', str(div[0])))
            text = soup_web.find_all("div",{"id":"jobDescriptionText"})
            lst = []
            
            for k in key_word:
                if re.search(k, str(text[0]), re.IGNORECASE):
                    lst.append(k)
            skill.append(lst)
            ## can adjust the sleep time based on your needs, but this step is essential to prevent you from being blocked
            sleep_time = random.randint(1,10)
            time.sleep(sleep_time)
    ## can adjust the sleep time based on your needs, but this step is essential to prevent you from being blocked        
    sleep_time = random.randint(20,60)
    time.sleep(sleep_time)

 94%|█████████████████████████████████████████████████████████████████████████████     | 31/33 [01:56<00:11,  5.79s/it]


100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [09:20<00:00, 140.16s/it]


#### data cleaning and removing records with drop words

In [4]:
DB = pd.DataFrame(
    {'Title': title,
     'Company': company,
     'Skill': skill,
     'Indeed.com Link': link})
DB['Skill'] = DB['Skill'].apply(lambda x: str(x)[2:-2])
DB['Skill'] = DB['Skill'].apply(lambda x: re.sub(r"'",'',x))
DB['Company'] = DB['Company'].apply(lambda x: str(x)[2:-2])

len1 = len(DB)

## drop records requiring key words listed below
lst_drop = ['U.S. Citizenship','contract']
for l in lst_drop:
    DB = DB[DB['Skill'].map(lambda x: l not in str(x))]
## drop rows with no match skills
DB = DB[DB['Skill'].map(lambda x: len(x) > 0)]
## drop records only match "Excel" skill
DB = DB[DB['Skill'].map(lambda x: str(x) != 'Excel')]

company_unique = list(set(DB['Company']))

company_reformat = []
for c in company_unique:
    if len(c.split()) > 1:
        company_reformat.append(c.replace(' ','+'))
    else:
        company_reformat.append(c)
        
company_link = []
for c in company_reformat:
    company_link.append('https://h1bdata.info/index.php?em='+c+'&job=&city=&year=All+Years')

#### scrape sponsorship records information from H1Bdata.info

In [5]:
record = []
for l in company_link:
    h1b_result = requests.get(l)
    h1b_src = h1b_result.content
    h1b_soup = BeautifulSoup(h1b_src,'html.parser')
    tbody = h1b_soup.find_all('tbody')
    if len(str(tbody)) < 20:
        record.append(0)
    else:
        number = re.findall(r'<tr>', str(tbody))
        record.append(len(number))    

#### data cleaning and dropping records with no historical sponsorship records

In [16]:
H1B = pd.DataFrame(
    {'Company': company_unique,
     '# of Sponsorship Record(s) from the Employer': record,
     'h1bdata.info Link': company_link})

## join two datasets
df = pd.merge(DB, H1B, on='Company', how='left')
## drop records with no historical sponsorship information
df = df[df['# of Sponsorship Record(s) from the Employer'].map(lambda x: x != 0)]
df.drop_duplicates(inplace = True)
len2 = len(df)

print(Fore.YELLOW + '# of records reduced - ' + Fore.GREEN + str(len1) + ' → ' + str(len2) + '（difference : ' + str(len1 - len2) + ')')

# of records reduced - 60 → 4（difference : 56)


#### final dataset

In [17]:
df

,Title,Company,Skill,Indeed.com Link,# of Sponsorship Record(s) from the Employer,h1bdata.info Link
0,Data Analyst,Midland Credit Management,"SQL, Power BI, Excel",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,91,https://h1bdata.info/index.php?em=Midland+Cred...
2,Data Analyst,Sagewell,"Analytics, Excel","https://www.indeed.com/company/Sagewell,-Inc./...",1,https://h1bdata.info/index.php?em=Sagewell&job...
4,Operations Data Analyst,ResMed,"SQL, Python, Tableau, Analytics, Statistics, E...",https://www.indeed.com/rc/clk?jk=bd3de1aff801a...,127,https://h1bdata.info/index.php?em=ResMed&job=&...
14,Data Analyst,Goby Inc,Model,https://www.indeed.com/rc/clk?jk=480dc36616e99...,12,https://h1bdata.info/index.php?em=Goby+Inc&job...


#### save data into csv file

In [18]:
df.to_csv('Job Info from Indeed.csv', index = False)